# Step 1: Importing Libraries..

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', 2000)
pd.set_option('display.max_rows', 500)

# Step 2: Dataset loading and Preprocessing..

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install patool
import patoolib
patoolib.extract_archive("/content/drive/MyDrive/ProAnalytics_G4/dataset_template_req/Insurance Dataset.rar", outdir="/content")

     |████████████████████████████████| 81kB 3.4MB/s 
patool: Extracting /content/drive/MyDrive/ProAnalytics_G4/dataset_template_req/Insurance Dataset.rar ...
patool: running /usr/bin/unrar x -- "/content/drive/MyDrive/ProAnalytics_G4/dataset_template_req/Insurance Dataset.rar"
patool:     with cwd='/content'
patool: ... /content/drive/MyDrive/ProAnalytics_G4/dataset_template_req/Insurance Dataset.rar extracted to `/content'.


'/content'

In [ ]:
dataset1 = pd.read_csv('./Insurance Dataset.csv')


In [ ]:
dataset.columns

Index(['Area_Service', 'Hospital County', 'Hospital Id', 'Age', 'Gender',
       'Cultural_group', 'ethnicity', 'Days_spend_hsptl', 'Admission_type',
       'Home or self care,', 'ccs_diagnosis_code', 'ccs_procedure_code',
       'apr_drg_description', 'Code_illness', 'Mortality risk',
       'Surg_Description', 'Weight_baby', 'Abortion', 'Emergency dept_yes/No',
       'Tot_charg', 'Tot_cost', 'ratio_of_total_costs_to_total_charges',
       'Result', 'Payment_Typology'],
      dtype='object')

In [ ]:
# dropping eda-2 columns

dataset.drop(['Hospital Id', 'apr_drg_description', 'Abortion'], axis=1, inplace=True)

In [ ]:
dataset.drop_duplicates(inplace=True)

In [ ]:
dataset.dropna(inplace=True)

### encoding...

#### Making it for deployment ppoint of view..

In [ ]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()

In [ ]:
label_area = label.fit_transform(dataset["Area_Service"])
label_Hospital_County = label.fit_transform(dataset["Hospital County"])
label_Age = label.fit_transform(dataset["Age"])
label_Cultural_group = label.fit_transform(dataset["Cultural_group"])
label_ethnicity= label.fit_transform(dataset["ethnicity"])
label_Admission_type = label.fit_transform(dataset["Admission_type"])
label_Home_or_self_care = label.fit_transform(dataset["Home or self care,"])

In [ ]:
from sklearn.preprocessing import LabelBinarizer
label_binizer=LabelBinarizer() 
one_hot_Gender = label_binizer.fit(dataset.Gender)
one_hot_Surg_Description = label_binizer.fit(dataset.Surg_Description)
one_hot_Emergency_dept_yes_No = label_binizer.fit(dataset["Emergency dept_yes/No"])

In [ ]:
one_hot_Gender.classes_

array(['N', 'Y'], dtype='<U1')

In [ ]:
label_encoder_list = [label_area, label_Hospital_County,
                label_Age, label_Cultural_group,
                label_ethnicity, label_Admission_type,
                label_Home_or_self_care]

one_hot_coder_list = [one_hot_Gender, one_hot_Surg_Description, one_hot_Emergency_dept_yes_No]

#### pickling encoding object & columns for deployment point of view...

In [ ]:
from joblib import dump, load

In [ ]:
dump(label_encoder_list, 'label_encoder_list.joblib')

['label_encoder_list.joblib']

In [ ]:
dump(one_hot_coder_list, 'one_hot_coder_list.joblib')

['one_hot_coder_list.joblib']

In [ ]:
label_to_column = ['Area_Service', 'Hospital_County',
                  'Age', 'Cultural_group', 'ethnicity',
                  'Admission_type', 'Home_or_self_care']

one_hot_column = ['Gender', 'Surg_Description', 'Emergency_dept_yes_No']

In [ ]:
dump(label_to_column, 'label_to_column.joblib')

In [ ]:
dump(one_hot_column, 'one_hot_column.joblib')

In [ ]:
# applyibng label encoding on columns..also this below cells can be used in deployment as well

In [ ]:
j = 0
for i in label_for:
    dataset[i] = label_encoder_list[j].fit_transform(dataset[i])
    j =+1

In [ ]:
# applyibng one hot encoding on columns..also this below cells can be used in deployment as well

In [ ]:
k = 0
for i in one_hot_for:
    dataset[i] = one_hot_coder_list[k].fit_transform(dataset[i])
    k =+1

In [ ]:
dataset.head(2)

,Area_Service,Hospital County,Age,Gender,Cultural_group,ethnicity,Days_spend_hsptl,Admission_type,"Home or self care,",ccs_diagnosis_code,ccs_procedure_code,Code_illness,Mortality risk,Surg_Description,Weight_baby,Emergency dept_yes/No,Tot_charg,Tot_cost,ratio_of_total_costs_to_total_charges,Result,Payment_Typology
0,Western NY,Allegany,30 to 49,F,White,Not Span/Hispanic,4,Elective,Home or Self Care,122,0,1,1.0,Medical,0,Y,5511.95,5582.49,1.012798,1,1
1,Western NY,Allegany,70 or Older,F,White,Not Span/Hispanic,4,Urgent,Short-term Hospital,197,0,3,2.0,Medical,0,Y,4783.20,5162.82,1.079365,1,1


# Step 3: Upsampling..& model 

In [ ]:
dataset = pd.read_csv('./processed_data_main.csv')


In [ ]:
dataset.drop(["Unnamed: 0"], axis=1,inplace=True)

In [ ]:
dataset["I15"].value_counts()

1.0    228551
0.0     76244
Name: I15, dtype: int64

In [ ]:
data_minority = dataset.loc[dataset['I15']==0]
data_majority = dataset.loc[dataset['I15']==1]

In [ ]:
dataset["I15"].value_counts()

1.0    228551
0.0     76244
Name: I15, dtype: int64

In [ ]:
data_minority.head()

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14,I15
3,-0.50,0.0,5.235848,1.0,0.00,0.0,1.0,0.0,1.0,0.180867,0.000000,0.000000,0.0,0.0,0.0,0.0
9,-0.25,0.0,3.316809,1.0,0.75,0.5,1.0,0.0,0.0,0.180867,0.333333,0.000000,0.0,0.0,0.0,0.0
11,-0.25,0.0,3.814060,1.0,0.75,0.5,1.0,0.0,0.0,0.180867,0.666667,0.666667,0.0,0.0,0.0,0.0
20,-0.50,0.0,4.478649,1.0,0.00,0.5,1.0,0.0,1.0,0.180867,0.000000,0.000000,0.0,0.0,0.0,0.0
21,-0.25,0.0,4.115207,1.0,0.75,0.0,1.0,0.0,0.0,0.216741,0.333333,0.000000,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.utils import resample
data_minority_upsampled = resample(data_minority, replace=True, n_samples=780361, random_state=42)

In [ ]:
data_upsampled = pd.concat([data_majority, data_minority_upsampled ], ignore_index=True)

In [ ]:
X = data_upsampled.drop(['I15'],axis=1).values
Y = data_upsampled['I15']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_RF_upsampled = RandomForestClassifier(n_estimators=10, random_state=42)

In [ ]:
model_RF_upsampled.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model_RF_upsampled.predict(X_train)

print(classification_report(Y_train, Y_pred))

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99    546252
         1.0       1.00      0.96      0.98    159986

    accuracy                           0.99    706238
   macro avg       0.99      0.98      0.99    706238
weighted avg       0.99      0.99      0.99    706238



In [ ]:
Y_pred_test = model_RF_upsampled.predict(X_test)

print(classification_report(Y_test, Y_pred_test))

              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94    234109
         1.0       1.00      0.59      0.74     68565

    accuracy                           0.91    302674
   macro avg       0.94      0.79      0.84    302674
weighted avg       0.92      0.91      0.90    302674



In [ ]:
print(confusion_matrix(Y_test, Y_pred_test))

[[233976    133]
 [ 28335  40230]]


In [ ]:
dataset.dropna(inplace=True)

In [ ]:
Y_pred_whole = model_RF_upsampled.predict(dataset.drop(['I15'],axis=1))

print(classification_report(dataset["I15"], Y_pred_whole))

              precision    recall  f1-score   support

         0.0       0.69      1.00      0.82     76244
         1.0       1.00      0.85      0.92    228551

    accuracy                           0.89    304795
   macro avg       0.85      0.92      0.87    304795
weighted avg       0.92      0.89      0.89    304795



In [ ]:
print(confusion_matrix(dataset["I15"], Y_pred_whole))

[[236476  24428]
 [ 85298 695063]]


In [ ]:

from sklearn.model_selection import StratifiedKFold 
# Create StratifiedKFold object. 
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1) 
lst_accu_stratified = [] 
   
for train_index, test_index in skf.split(X, Y): 
    x_train_fold, x_test_fold = X[train_index], X[test_index] 
    y_train_fold, y_test_fold = Y[train_index], Y[test_index] 
    model_RF_upsampled.fit(x_train_fold, y_train_fold) 
    lst_accu_stratified.append(model_RF_upsampled.score(x_test_fold, y_test_fold)) 
   
# Print the output. 
print('List of possible accuracy:', lst_accu_stratified) 
print('\nMaximum Accuracy That can be obtained from this model is:', 
      max(lst_accu_stratified)*100, '%') 
print('\nMinimum Accuracy:', 
      min(lst_accu_stratified)*100, '%') 
print('\nOverall Accuracy:', 
      np.mean(lst_accu_stratified)*100, '%') 
print('\nStandard Deviation is:', np.std(lst_accu_stratified))

List of possible accuracy: [0.8396647722540094, 0.8393444093469081, 0.8375172997078272, 0.8408426879901584, 0.839542006253524, 0.8397470398277718, 0.8407081347070583, 0.8408234660925726, 0.8395227843559383, 0.8405863960223486]

Maximum Accuracy That can be obtained from this model is: 84.08426879901583 %

Minimum Accuracy: 83.75172997078272 %

Overall Accuracy: 83.98298996558117 %

Standard Deviation is: 0.0009566427034243988


In [ ]:
from joblib import dump, load
dump(model_RF_upsampled, 'model_RF_upsampled.joblib') 

['model_RF_upsampled.joblib']